# Preparing the data

##Importing libraries, downloading the model

In [ ]:
import pandas
import sklearn
import numpy
import spacy
import string
import sys
from sklearn.linear_model import LogisticRegression
import pickle
from sklearn.metrics import accuracy_score, classification_report

print(pandas.__version__)
print(sklearn.__version__)
print(numpy.__version__)

1.5.3
1.2.2
1.22.4


In [ ]:
# Small Russian model:
# !python -m spacy download ru_core_news_sm
# nlp = spacy.load('ru_core_news_sm')

# Large Russian model:
!python -m spacy download ru_core_news_lg
nlp = spacy.load('ru_core_news_lg')

2023-06-15 21:40:28.268684: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 55.6 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=694fa8f6d504ff892c5cf2d558253b1e53e5b793d81c00fa4944bf36c17c9fc8
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')


##Making lists and doc objects from csv files

In [ ]:
# Let's load the training data from a csv file
train_set = pandas.read_csv('./train_data.csv', encoding='utf-8')
# train_set

In [ ]:
test_set = pandas.read_csv('./test_data.csv', encoding='utf-8')
# test_set

In [ ]:
train_sentences = train_set['text'].to_list()
train_authors = train_set['author'].to_list()

test_sentences = test_set['text'].to_list()
test_authors = test_set['author'].to_list()

print(len(train_authors), len(test_authors))

10000 1000


In [ ]:
train_doc_sentences = nlp.pipe(train_sentences)
test_doc_sentences = nlp.pipe(test_sentences)

## Preparing the feature set

First we will import the Latin letters using 'string.ascii_letters'.

In [ ]:
print(string.ascii_letters)

abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ


Then we will check if additionaly French, German or Ukrainian special characters appear in the sentences, since all four writers were known to be using words from these languages.

In [ ]:
french_accent_marks = "éèêëàÉÈÊËÀ"
german_characters = "äöüßÄÖÜ"
ukrainian_characters = "ґєїіҐЄЇІ"

for sentence in train_sentences:
  for character in sentence:
    if character in ukrainian_characters:
      print('Ukr character found')
    if character in german_characters:
      print('German character found')
    if character in french_accent_marks:
      print('French character found')

French character found
French character found
French character found
French character found
French character found
French character found
French character found
French character found
French character found
French character found
French character found
French character found
French character found
French character found
French character found
French character found
French character found
French character found
French character found
French character found
French character found
French character found
French character found
French character found
French character found
French character found
French character found
French character found
French character found
French character found
French character found
French character found
French character found


As we can see, there are no special Ukrainian or German characters, but there are special French characters. Thus we will add them to our character string.

In [ ]:
new_string = string.ascii_letters + french_accent_marks
print(len(new_string), new_string)

62 abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZéèêëàÉÈÊËÀ



In [ ]:
# We are creating a matrix with zero vectors for each review (in training set and test set)
train_features_matrix = numpy.zeros((len(train_sentences), len(new_string)))
print(train_features_matrix.shape)

test_features_matrix = numpy.zeros((len(test_sentences), len(new_string)))
print(test_features_matrix.shape)

(10000, 62)
(1000, 62)


# Modifying the feature vectors

## Visualisation tests

In [ ]:
counter = 0

for sentence, author in zip(train_sentences, train_authors):
    for char in sentence:
      if char in new_string:
        print('Author:', author)
        print(sentence)
        char_id = new_string.index(char)
        print(char_id)
        break
    counter +=1
    if counter == 40:
      break
        # sys.exit()

Author: Tolstoy
Oh, sans doute, c'est la plus charmante femme du monde, — сказала Анна Павловна с улыбкой над своей восторженностью.
40


Visualisation test 2:

In [ ]:
counter = 0

for sentence, author, feature_vector in zip(train_sentences, train_authors, train_features_matrix):
    for char in sentence:
      if char in new_string:
        print('Author:', author)
        print(sentence)
        char_id = new_string.index(char)
        print(char_id)
        feature_vector[char_id] = 1
        print(feature_vector.tolist())
        # sys.exit()
    counter +=1
    if counter == 40:
      break

## Writing a function for vector modification

In [ ]:
def modify_feature_vectors(sentences, features_matrix):
  for sentence, feature_vector in zip(sentences, features_matrix):
    for char in sentence:
      if char in new_string:
        char_id = new_string.index(char)
        feature_vector[char_id] = 1
  return features_matrix

In [ ]:
train_features_matrix = numpy.zeros((len(train_sentences), len(new_string)))
train_features_matrix_final = modify_feature_vectors(train_sentences, train_features_matrix)

print(train_features_matrix_final[35:40])

# Training

In [ ]:
lr_latin = LogisticRegression()

# Train the model on the data, storing the information learned from the dat`a
# Model is learning the relationship between digits (x_train) and labels (y_train)
lr_latin.fit(train_features_matrix_final, train_authors)

print(lr_latin.classes_)
print(lr_latin.get_params())

['Chekhov' 'Dostoevsky' 'Gogol' 'Tolstoy']
{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Modifying the test set feature vectors

In [ ]:
test_features_matrix = numpy.zeros((len(test_sentences), len(new_string)))
test_features_matrix_final = modify_feature_vectors(test_sentences, test_features_matrix)

print(test_features_matrix_final[35:40])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


# Making predictions

In [ ]:
def predict(i):
    print(test_sentences[i])
    # print the features of the index
    print(test_features_matrix_final[i])
    # print the correct label of the index
    print(test_authors[i])

    print()
    print("Prediction:")
    # print the prediction for the features of this index
    print(lr_latin.predict([test_features_matrix_final[i]]))
    # print the probabilities for each label predictions
    print(lr_latin.predict_proba([test_features_matrix_final[i]]))
    print()

In [ ]:
predict(0)
predict(28)
predict(39)

"Фома Фомич, говорю, разве это возможное дело?
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Dostoevsky

Prediction:
['Chekhov']
[[0.25429837 0.25096826 0.25394822 0.24078515]]

Je vous dois la vie.
[1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Tolstoy

Prediction:
['Tolstoy']
[[0.0466672  0.34852971 0.00792362 0.59687947]]

На одной скамье, в уединенной аллее, увидел я m-me M *.
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Dostoevsky

Prediction:
['Chekhov']
[[0.37399333 0.2254538  0.09022878 0.31032409]]



# Saving the model

In [ ]:
# Save to file in the current working directory
pkl_filename = "logreg_latin.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(lr_latin, file)

# Evaluating the model

## Dummy Model

In [ ]:
dummy_predictions = ['Dostoevsky'] * len(test_sentences)
print(len(dummy_predictions))

# Calculate the accuracy of these "dummy predictions"
acc_dummy = accuracy_score(test_authors, dummy_predictions)
print(f'The accuracy is: {acc_dummy}')
print()

print(classification_report(test_authors, dummy_predictions))

1000
The accuracy is: 0.25

              precision    recall  f1-score   support

     Chekhov       0.00      0.00      0.00       250
  Dostoevsky       0.25      1.00      0.40       250
       Gogol       0.00      0.00      0.00       250
     Tolstoy       0.00      0.00      0.00       250

    accuracy                           0.25      1000
   macro avg       0.06      0.25      0.10      1000
weighted avg       0.06      0.25      0.10      1000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Latin characters model

In [ ]:
print('Accuracy:')

acc = accuracy_score(test_authors, test_predictions_latin)
print(acc)
corr_count = accuracy_score(test_authors, test_predictions_latin, normalize=False)
total_count = len(test_authors)

print(f'Total reviews: {str(total_count)}')
print(f'Total correct predictions: {str(corr_count)}')
corr_ratio = corr_count / total_count
print(f'Correct ratio: {str(corr_ratio)}')

Accuracy:
0.265
Total reviews: 1000
Total correct predictions:265
Correct ratio:0.265


In [ ]:
print(classification_report(test_authors, test_predictions_latin))

              precision    recall  f1-score   support

     Chekhov       0.25      0.99      0.40       250
  Dostoevsky       0.50      0.01      0.02       250
       Gogol       0.00      0.00      0.00       250
     Tolstoy       0.70      0.06      0.12       250

    accuracy                           0.27      1000
   macro avg       0.36      0.27      0.13      1000
weighted avg       0.36      0.27      0.13      1000

